# LSTM Learning with TensorFlow

We will demonstrate fitting a recurrent neural network to a text file.

Based on https://github.com/skyer9/hello_lstm, ported to Jupyter notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell, GRUCell
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.seq2seq import sequence_loss

import numpy as np

In [2]:
#X_train = "Hello, World!"
# X_train = "안녕하세요, World!"
X_train = ("if you want to build a ship, don't drum up people together to "
           "collect wood and don't assign them tasks and work, but rather "
           "teach them to long for the endless immensity of the sea.")

#nb_epoches = 25
nb_epoches = 100
batch_size = 1
num_steps = 100 # Length of gradient unchaining

In [3]:
with open(r'd:\projects\gltestplus\src\gltestplus.cpp', encoding='utf-8') as f:
    text = f.read()
    #text = f.readlines()

X_train = text[:1000].replace('\r', '').replace('\n', '')
print(len(X_train))

960


In [4]:
def create_vocabulary(sequence):
    vocab = {}
    for i in range(len(sequence)):
        ch = sequence[i]
        if ch in vocab:
            vocab[ch] += 1
        else:
            vocab[ch] = 1
    vocab_rev = sorted(vocab, key=vocab.get, reverse=True)
    vocab = dict([(x, y) for (y, x) in enumerate(vocab_rev)])
    return vocab, vocab_rev


def sentence_to_token_ids(sentence, vocabulary):
    characters = [sentence[i:i+1] for i in range(0, len(sentence), 1)]
    return [vocabulary.get(w) for w in characters]


def token_ids_to_one_hot(token_ids, num_classes=10):
    token_ids_one_hot = np.zeros((len(token_ids), num_classes))
    token_ids_one_hot[np.arange(len(token_ids)), token_ids] = 1
    return token_ids_one_hot



In [5]:
sequence_length = len(X_train) - 1

X_train_vocab, X_train_vocab_rev = create_vocabulary(X_train)
hidden_size = len(X_train_vocab)
num_classes = len(X_train_vocab)

X_train_ids = sentence_to_token_ids(X_train, X_train_vocab)
X_data = X_train_ids[:-1]
Y_data = X_train_ids[1:]
X_data_one_hot = [token_ids_to_one_hot(X_data, num_classes)]
Y_data = [Y_data]



In [6]:
used = False

with tf.name_scope('input_X'):
    X = tf.placeholder(tf.float32, [None, sequence_length, hidden_size], name='X')

with tf.name_scope('output_Y'):
    Y = tf.placeholder(tf.int32, [None, sequence_length], name='Y')

with tf.name_scope('lstm'):
    #cell = BasicLSTMCell(num_units=hidden_size)
    cell = GRUCell(num_units=hidden_size)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell,
                                         X,
                                         initial_state=initial_state,
                                         dtype=tf.float32)
    tf.summary.histogram('rnn_outputs', outputs)

    X_for_fc = tf.reshape(outputs, [-1, hidden_size])
    outputs = fully_connected(inputs=X_for_fc,
                              num_outputs=num_classes,
                              activation_fn=None)

    outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
    tf.summary.histogram('outputs', outputs)

    weights = tf.ones([batch_size, sequence_length])
    tf.summary.histogram('weights', weights)
with tf.name_scope('sequence_loss'):
    sloss = sequence_loss(logits=outputs,
                                  targets=Y,
                                  weights=weights)
    tf.summary.scalar('sequence_loss', sloss)
with tf.name_scope('loss'):
    loss = tf.reduce_mean(sloss)
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

with tf.name_scope('prediction'):
    prediction = tf.argmax(outputs, axis=2)
    
used = True
print(cell)

In [7]:
print(X_data_one_hot[0].shape, len(Y_data[0]), Y)

(959, 62) 959 Tensor("output_Y/Y:0", shape=(?, 959), dtype=int32)


In [11]:
GRUCell?

In [10]:
tf.strided_slice?

In [15]:
sequence_loss?

In [8]:
sess = tf.Session()

In [11]:
fileWriter = tf.summary.FileWriter('TensorFlowLSTM/2')
fileWriter.add_graph(sess.graph)

In [12]:
merged_summary = tf.summary.merge_all()

In [13]:
nb_epoches = 1000
loss_hist = []

sess.run(tf.global_variables_initializer())
for i in range(nb_epoches):

    if False:
        in2 = tf.train.range_input_producer(nb_epoches, shuffle=False).dequeue()  # [0, 1, .., epoch_size-1] という整数を順ぐりに無限生成するイテレータ
        x = tf.strided_slice(X_data_one_hot[0], [0, in2 * num_steps],
                             [batch_size, (in2 + 1) * num_steps])  # この使われ方の strided_slice は、data[0:batch_size, i*num_steps:(i+1)*num_steps] だと思って良い
        x.set_shape([batch_size, num_steps])
        y = tf.strided_slice(Y_data[0], [in2 * num_steps],
                             [(in2 + 1) * num_steps])  # 正解 y は x の次に来る単語なので、1を足してスライスを右に一つずらす
        y.set_shape([num_steps])
    else:
        x = X_data_one_hot
        y = Y_data

    l, _ = sess.run([loss, train], feed_dict={X: x, Y: y})
    result = sess.run(prediction, feed_dict={X: x})
    print(i, "loss:", l) #, "prediction: ", result, "true Y: ", Y_data)
    s = sess.run(merged_summary, feed_dict={X: x, Y: y})
    fileWriter.add_summary(s, i)
    if np.isnan(l):
        print("NaN detected. Possibly floating overflow.")
        break
    result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
    if i % 10 == 0:
        print("\tPrediction str: ", ''.join(result_str))
    loss_hist.append(l)

0 loss: 4.12632
	Prediction str:  ######i##iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
1 loss

71 loss: 1.02122
72 loss: 0.976957
73 loss: 0.953388
74 loss: 0.956743
75 loss: 0.933628
76 loss: 0.927428
77 loss: 0.909619
78 loss: 0.866512
79 loss: 0.849611
80 loss: 0.8383
	Prediction str:  ** \file  * \breef The matn m wdee file for mlees rlo  minM ws (luart  * * The mnt  iptin  fir mhe mnoge me ni matie  matn() or iin(ain(), resone  tn(mhen fine  * *t de le *in  fhe matn me itml ms m, mpn tL inetialinatitnt hni fvent matting  * *oma meaminc ma honi *pi mts  de inc i/en s * giefine mSEWIN 1 ///g 1herher mde winc ws eri (wul*)#if !USEWINVinclude <wLWglutih"#i eeninclude "gntillgteh"#include "gppltcltinn.h"#iefinc UINVER 0x0500#iefinc wWINV2_WINNT 0x0500#include "winc ws.h>#indea#include "game.h"#include "ginniryh"#include "gl.yer.h"#include "gnt.or.h"#include "goorcSyi-h"#include "goorcSys-finc.h"#include "gte.e.rEfile.h"#include "gtti.weawih"#include "gtirEnud.h"#include "glc.h"#include "geybinc.h"#include "gotpon.h"#include "glw/mlwincow.h"#include "glw/GLWce dd "#include "glw/G

145 loss: 0.206828
146 loss: 0.202312
147 loss: 0.196875
148 loss: 0.190658
149 loss: 0.18326
150 loss: 0.178219
	Prediction str:  ** \file  * \brief The main source file for glsestplus Windows client. * * The entry point for the prograt, no mateer main() or WinMain(), resides in this file  * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methode are also define  here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "Antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Clayer.h"#include "Cntity.h"#include "CoordSys.h"#include "goordSys-find.h"#include "stellar_file.h"#include "sltrodraw.h"#include "StarEnum.h"#include "Cmd.h"#include "keybind.h"#include "gotion.h"#include "glw/glwindow.h"#include "glw/GLWcenu.h"#include "glw/GLWchart.h"#include "docker.h"#include "draw/material.h"//#includ

218 loss: 0.110406
219 loss: 0.108912
220 loss: 0.103755
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entryipoint for the program, no matter main() or WinMain(), resides in th s file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also define  here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Cntity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "sstrodraw.h"#include "StarEnum.h"#include "Cmd.h"#include "keybind.h"#include "motion.h"#include "glw/Glwindow.h"#include "glw/GLWmenu.h"#include "glw/GLWmhart.h"#include "Docker.h"#include "sraw/material.h"//#include "Sceptor.h"#include "glslack.h"#include "Universe.h"#i

287 loss: 0.183187
288 loss: 0.172724
289 loss: 0.174221
290 loss: 0.160872
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows cluent. * * The entry point for the program, no mateer main() on WinMain(), resides in th s file. * It deals with the main sessege loop, mpenGL inatialization, and hvent mapping. * Some drawing methods ara also define  he e. */#define USEWIN 1 ///< whether use windows api (wgl*)#in !USEWIN#include <GL/glut.h>#elst#include "gntiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WIN.T 0x0500#include <windows.h>#endif#include "Vame.h"#include "Viewer.h"#include "Player.h"#include "Cntity.h"#include "CoordSy..h"#include "CoordSys-find.h"#include "ste.lar_file.h"#include "astrodrawsh"#include "StarEnum.h"#include "Cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindowuh"#include "glw/GLWmenu.h"#include "glw/GLWmhart.h"#include "Docker.h"#include "dr w/materia/.h"//#include "Sceptor.h"#include "glstawk.h"#inc

...

701 loss: 0.00285215
702 loss: 0.00284565
703 loss: 0.00284574
704 loss: 0.00282767
705 loss: 0.00281577
706 loss: 0.00280562
707 loss: 0.00279656
708 loss: 0.00278865
709 loss: 0.00278125
710 loss: 0.0027762
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#includ

771 loss: 0.00231848
772 loss: 0.0023122
773 loss: 0.00230484
774 loss: 0.00229836
775 loss: 0.00229136
776 loss: 0.00228484
777 loss: 0.00227837
778 loss: 0.00227216
779 loss: 0.00226605
780 loss: 0.00226034
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#includ

841 loss: 0.0019355
842 loss: 0.00193075
843 loss: 0.00192643
844 loss: 0.00192135
845 loss: 0.00191594
846 loss: 0.00191109
847 loss: 0.0019062
848 loss: 0.00190181
849 loss: 0.00189693
850 loss: 0.00189241
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#include

911 loss: 0.00164915
912 loss: 0.00164547
913 loss: 0.00163882
914 loss: 0.00163939
915 loss: 0.00163637
916 loss: 0.00162798
917 loss: 0.00162977
918 loss: 0.00162605
919 loss: 0.00161858
920 loss: 0.00162189
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#inclu

981 loss: 0.00142153
982 loss: 0.00141818
983 loss: 0.00141539
984 loss: 0.00141183
985 loss: 0.00140959
986 loss: 0.00140621
987 loss: 0.00140356
988 loss: 0.00140074
989 loss: 0.00139808
990 loss: 0.00139597
	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#inclu

In [9]:
saver = tf.train.Saver()

In [ ]:
save_path = saver.save(sess, "TensorFlowLSTM/model.ckpt")
print("Model saved in file: %s" % save_path)

In [10]:
saver.restore(sess, "TensorFlowLSTM/model.ckpt")

INFO:tensorflow:Restoring parameters from TensorFlowLSTM/model.ckpt


In [16]:
result = sess.run(prediction, feed_dict={X: X_data_one_hot})
result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
print("\tPrediction str: ", ''.join(result_str))
print("\tTruth str: ", ''.join([X_train_vocab_rev[c] for c in np.squeeze(Y_data)]))

	Prediction str:  ** \file  * \brief The main source file for gltestplus Windows client. * * The entry point for the program, no matter main() or WinMain(), resides in this file. * It deals with the main message loop, OpenGL initialization, and event mapping. * Some drawing methods are also defined here. */#define USEWIN 1 ///< whether use windows api (wgl*)#if !USEWIN#include <GL/glut.h>#else#include "antiglut.h"#include "Application.h"#define WINVER 0x0500#define _WIN32_WINNT 0x0500#include <windows.h>#endif#include "Game.h"#include "Viewer.h"#include "Player.h"#include "Entity.h"#include "CoordSys.h"#include "CoordSys-find.h"#include "stellar_file.h"#include "astrodraw.h"#include "StarEnum.h"#include "cmd.h"#include "keybind.h"#include "motion.h"#include "glw/glwindow.h"#include "glw/GLWmenu.h"#include "glw/GLWchart.h"#include "Docker.h"#include "draw/material.h"//#include "Sceptor.h"#include "glstack.h"#include "Universe.h"#include "glsl.h"#include "sqadapt.h
	Truth str:  ** \file 

In [30]:
result = sess.run(prediction, feed_dict={X: [token_ids_to_one_hot(np.random.randint(max(X_data), size=(sequence_length)), num_classes)]})
for i in range(10):
    result = sess.run(prediction, feed_dict={X: [token_ids_to_one_hot(np.array(result).flat, num_classes)]})
    result_str = [X_train_vocab_rev[c] for c in np.squeeze(result)]
    print("\tPrediction str: ", ''.join(result_str))

	Prediction str:  wliiusondrUSVnnwep hIi  nwete.tatd gloteipmlidee hilLfthmarlieaanrulgh*dnfh pt.lntoipiptiuanoheen hogU_lhflte._e#tweamincapleampiennNNiegignrihmht#iadleihiinl hvrersl#GuarUuaph* hleeesisln .mEhe/tlsairlhta glShehuinghSlzmih#hamgih# > ia3 /iaihfw_NigoihwrhinigigirllS"ehp uln lirpstgiehir phpdlp.hnUplhaaip roeipUdphpltgSa pp/  ls lw/gnenlh ihfpahpshp/ ipo pSfhlaag pgiptein wUaoh/hppi  tthsiph /iSpnhghg.hplh*lur5newi.orelien .l pte /nghlpoaehtieors h    h/ln gvip/tplu p/rgeaeeagaeagpgn p  fpli svplelpigal.i ppghp pvdtt/UrepteghppnmhvhppuflhlofepgUaa"ilggnrrlnolL.irlerssheghnesoslUroanelife lehse shW v g/ireowawv  e/aviialaieehhhlni"/iihn/aee aSp eeepoaapuihlhehlui/lhwinglhliG/hiigppnGgindrfwan sh//. luilhppiedUh /gaeerUd//gpihpappehapU lnphtpossipsnedesppepU  " /glihhphioeh iSlugceipU/   p/ pSpphip.rgSrodigEWIpIeeiaellhowaUac .hti/he pmhgtigie iee p a"Siaileahgihp/  e.plplt/ pgggggirguapEheieoeh#. h#."tse#tntedfh eSieiWn .o u lgetomhihihgnheeehg"p/
	Prediction str:  iiot